# Intro

The goal of this notebook is to extend upon the work of [MANOJ PRABHAKAR](https://www.kaggle.com/manojprabhaakr) in the [EFFNET B6 WHALE COMP](https://www.kaggle.com/manojprabhaakr/effnet-b6-whale-comp), utilising the [Detic bounding boxes](https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/305503) to precrop the images.

I have created a new set of TFRecords which includes the bounding boxes in [this](https://www.kaggle.com/lextoumbourou/happywhale-tfrecords-with-bounding-boxes) repo.

At the time of creation, this notebook achieves a top 100 score. Given there's still 2 months left in the competition and that this data is all publically available, I felt it to be reasonable to release.

# Google drive connection

In [ ]:
#effv2
# https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/307995
# !pip install kaggle_datasets
# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH=KaggleDatasets().get_gcs_path('efficientnetv2-tfhub-weight-files')
# print('GCS_DS_PATH', GCS_DS_PATH)
EFFNETV2_ROOT = 'gs://kds-b9810ad22af72f1e7eacba31a60bffc027d24e6ec7c54b863acea0d3'



In [ ]:
from kaggle_datasets import KaggleDatasets
#small val datasets
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv10valval')
# GCS_DS_PATH gs://kds-4e353a1a78207392ebd5556217aa8fa5b6c7c68e656b23ac0dffc999
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv105valval')
# GCS_DS_PATH gs://kds-dd609279d91a722b5abf450bb5d97e83983045fcc1e36ef16c665a17
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv11valval')
# GCS_DS_PATH_VAL = 'gs://kds-17738bb688029b9512a51f01ae2807cf1e6101a9beca572a9d360697'
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv10val')
# GCS_DS_PATH gs://kds-43f32c5ce3c6907085249e42958ada0574574303970d4786c4b81635
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv105val')
# GCS_DS_PATH gs://kds-1b1813a3ca80ffbd81c5e4f43410ea012569fc3f47f0238d005f6a8d
# #GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv11val')
# GCS_DS_PATH = 'gs://kds-1fe71031b93156a76a3db9e65877e9e9b960810e79ff4f1997e39ea2'

GCS_DS_PATH=KaggleDatasets().get_gcs_path('vbodyv10vf8')
# GCS_DS_PATH = 'gs://kds-8f7821012e27f398d3a41dc4abe6afdb308dadc13352daa3cb2c83b5'

#Pseudo
GCS_DS_PATH_PSEUDO=KaggleDatasets().get_gcs_path('vbodyv10addvfinal')
# GCS_DS_PATH_PSEUDO = 'gs://kds-f14272d94700b908b711fa4acee2ef96ec88f23c71e26fcacf6fb98a'
#GCS_DS_PATH=KaggleDatasets().get_gcs_path('dorsalfinv10addv3')
#GCS_DS_PATH gs://kds-111a945510244e5eb5b3bca82d31a3c3dd7de0bbbd3fc5868470c38d


In [ ]:
# # FEATURE_VECTOR = f'{EFFNETV2_ROOT}/tfhub_models/efficientnetv2-{config.EFF_NETV2}/feature_vector'
# train_files = np.sort(np.array(tf.io.gfile.glob(FEATURE_VECTOR + '/*.*')))
# train_files

In [ ]:
# unzip the download dataset
# !mkdir /kaggle
# !mkdir /kaggle/input
# !mkdir /kaggle/input/happywhale-splits
# !mkdir /kaggle/logs
# !mkdir /kaggle/models


In [ ]:
import tensorflow as tf
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# import os
# from tensorflow.python.profiler import profiler_client

# tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
# print(profiler_client.monitor(tpu_profile_service_address, 100, 2))

In [ ]:
!pip install -q efficientnet
!pip install tensorflow_addons
import re
import os
import numpy as np
import pandas as pd
import random
import math
import tensorflow as tf
import efficientnet.tfkeras as efn
from sklearn import metrics
from sklearn.model_selection import KFold, train_test_split
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pickle
import json
import tensorflow_hub as tfhub
from datetime import datetime

## Config

In [ ]:
train_files = np.sort(np.array(tf.io.gfile.glob(GCS_DS_PATH + '/happywhale-2022-train*.tfrec')))
train_files

In [ ]:
class config:
    
    
    SEED = 42
    FOLD_TO_RUN = 10 #0
    FOLDS = 11 #5
    DEBUG = False
    EVALUATE = True
    RESUME = False
    RESUME_EPOCH = 9
    
    
    ### Dataset
    BATCH_SIZE = 8 * strategy.num_replicas_in_sync
    IMAGE_SIZE = 1024 #608
    N_CLASSES = 15587
    
    ### Model
    model_type = 'effnetv1' #'effnetv1'  
    EFF_NET = 5
    EFF_NETV2 = 'xl-21k-ft1k' #'s-21k-ft1k'
    FREEZE_BATCH_NORM = False
    head = 'arcface' 
    EPOCHS = 3
    LR = 0.001
    message='baseline'
    
    ### Augmentations
    CUTOUT = False
    
    ### Save-Directory
    save_dir = '.'#save_dir
    
    ### Inference
    KNN = 100
    
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def is_interactive():
    return 'runtime'    in get_ipython().config.IPKernelApp.connection_file
IS_INTERACTIVE = is_interactive()
print(IS_INTERACTIVE)

In [ ]:
MODEL_NAME = None
if config.model_type == 'effnetv1':
    MODEL_NAME = f'effnetv1_b{config.EFF_NET}'
elif config.model_type == 'effnetv2':
    MODEL_NAME = f'effnetv2_{config.EFF_NETV2}'

config.MODEL_NAME = MODEL_NAME
print(MODEL_NAME)

In [ ]:
save_dir = '.'
# save_dir = '/content/gdrive/MyDrive/kaggle/Whale/output/temp'

EXPERIMENT = 1
# run_ts = datetime.now().strftime('%Y%m%d-%H%M%S')
# print(run_ts)
# run_ts = run_ts + MODEL_NAME + '_rev' + str(config.IMAGE_SIZE)
# if IS_COLAB:
#     save_dir = f'/content/gdrive/MyDrive/kaggle/Whale/output/{run_ts}'
#     !mkdir -p {save_dir}

print(save_dir)

In [ ]:
with open(config.save_dir+'/config.json', 'w') as fp:
    json.dump({x:dict(config.__dict__)[x] for x in dict(config.__dict__) if not x.startswith('_')}, fp)

In [ ]:
train_files = np.sort(np.array(tf.io.gfile.glob(GCS_DS_PATH + '/happywhale-2022-train*.tfrec')))
test_files = np.sort(np.array(tf.io.gfile.glob(GCS_DS_PATH + '/happywhale-2022-test*.tfrec')))
# val_files = np.sort(np.array(tf.io.gfile.glob(GCS_DS_PATH_VAL + '/happywhale-2022-train*.tfrec')))
pseudo_files = np.sort(np.array(tf.io.gfile.glob(GCS_DS_PATH_PSEUDO + '/happywhale-2022-train*.tfrec')))
print(GCS_DS_PATH)
# print(GCS_DS_PATH_VAL)
print(GCS_DS_PATH_PSEUDO)
print(len(train_files),len(test_files),count_data_items(train_files),count_data_items(test_files))
# print(len(train_files),len(val_files),len(test_files),count_data_items(train_files),count_data_items(val_files),count_data_items(test_files))
print(len(pseudo_files),count_data_items(pseudo_files))

## Data

In [ ]:
SATURATION  = (0.9, 1.1)
CONTRAST = (0.9, 1.1)
BRIGHTNESS  =  0.1
ROTATION    = 10.0
SHEAR    = 10.0
HZOOM  = 8.0
WZOOM  = 4.0
HSHIFT = 4.0
WSHIFT = 4.0

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = config.IMAGE_SIZE
    XDIM = DIM%2
    
    rot = ROTATION * tf.random.normal([1], dtype='float32')
    shr = SHEAR * tf.random.normal([1], dtype='float32')
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM
    h_shift = HSHIFT * tf.random.normal([1], dtype='float32')
    w_shift = WSHIFT * tf.random.normal([1], dtype='float32')
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot, shr, h_zoom, w_zoom, h_shift, w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
def arcface_format(posting_id, image, label_group, matches):
    return posting_id, {'inp1': image, 'inp2': label_group}, label_group, matches

def arcface_inference_format(posting_id, image, label_group, matches):
    return image,posting_id

def arcface_eval_format(posting_id, image, label_group, matches):
    return image,label_group

# Data augmentation function
def data_augment(posting_id, image, label_group, matches):

    ### CUTOUT
    if tf.random.uniform([])>0.5 and config.CUTOUT:
      N_CUTOUT = 6
      for cutouts in range(N_CUTOUT):
        if tf.random.uniform([])>0.5:
           DIM = config.IMAGE_SIZE
           CUTOUT_LENGTH = DIM//32
           x1 = tf.cast( tf.random.uniform([],0,DIM-CUTOUT_LENGTH),tf.int32)
           x2 = tf.cast( tf.random.uniform([],0,DIM-CUTOUT_LENGTH),tf.int32)
           filter_ = tf.concat([tf.zeros((x1,CUTOUT_LENGTH)),tf.ones((CUTOUT_LENGTH,CUTOUT_LENGTH)),tf.zeros((DIM-x1-CUTOUT_LENGTH,CUTOUT_LENGTH))],axis=0)
           filter_ = tf.concat([tf.zeros((DIM,x2)),filter_,tf.zeros((DIM,DIM-x2-CUTOUT_LENGTH))],axis=1)
           cutout = tf.reshape(1-filter_,(DIM,DIM,1))
           image = cutout*image

    image = tf.image.random_flip_left_right(image)
    # image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    # image = tf.image.random_saturation(image, SATURATION[0], SATURATION[1])
    image = tf.image.random_contrast(image, 0.80, 1.20)
    # image = tf.image.random_contrast(image, CONTRAST[0], CONTRAST[1])
    image = tf.image.random_brightness(image, 0.10)
    # image = tf.image.random_brightness(image, BRIGHTNESS)
    if tf.random.uniform([])>0.5:
        image = transform(image)
    return posting_id, image, label_group, matches

# Function to decode our images
# Updated to include crops.
# def decode_image(image_data, box):
#     if box is not None and box[0] != -1:
#         left, top, right, bottom = box[0], box[1], box[2], box[3]
#         bbs = tf.convert_to_tensor([top, left, bottom - top, right - left])
#         image = tf.io.decode_and_crop_jpeg(image_data, bbs, channels=3)
#     else:
#         image = tf.image.decode_jpeg(image_data, channels = 3)

#     image = tf.image.resize(image, [config.IMAGE_SIZE,config.IMAGE_SIZE])
#     image = tf.cast(image, tf.float32) / 255.0
#     return image
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, [config.IMAGE_SIZE,config.IMAGE_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    return image

# This function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.int64),
        # 'detic_box': tf.io.FixedLenFeature([4], tf.int64),
         # 'yolov5_box': tf.io.FixedLenFeature([4], tf.int64),
    }

    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    posting_id = example['image_name']
    # bb = tf.cast(example['detic_box'], tf.int32)
    # image = decode_image(example['image'], bb)
    image = decode_image(example['image'])
#     label_group = tf.one_hot(tf.cast(example['label_group'], tf.int32), depth = N_CLASSES)
    label_group = tf.cast(example['target'], tf.int32)
#     matches = example['matches']
    matches = 1
    return posting_id, image, label_group, matches

# This function loads TF Records and parse them into tensors
def load_dataset(filenames, ordered = False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
#     dataset = dataset.cache()
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function is to get our training tensors
def get_training_dataset(filenames):
    dataset = load_dataset(filenames, ordered = False)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(config.BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# This function is to get our training tensors
def get_val_dataset(filenames):
    dataset = load_dataset(filenames, ordered = True)
    # dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))
    dataset = dataset.batch(config.BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# This function is to get our training tensors
def get_eval_dataset(filenames, get_targets = True):
    dataset = load_dataset(filenames, ordered = True)
    # dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_eval_format, num_parallel_calls = AUTO)
    if not get_targets:
        dataset = dataset.map(lambda image, target: image)
    dataset = dataset.batch(config.BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# This function is to get our training tensors
def get_test_dataset(filenames, get_names = True):
    dataset = load_dataset(filenames, ordered = True)
    # dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_inference_format, num_parallel_calls = AUTO)
    if not get_names:
        dataset = dataset.map(lambda image, posting_id: image)
    dataset = dataset.batch(config.BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
# row = 10; col = 8;
# row = min(row,config.BATCH_SIZE//col)
# N_TRAIN = count_data_items(train_files)
# print(N_TRAIN)
# ds = get_training_dataset(train_files)

# for (sample,label) in ds:
#     img = sample['inp1']
#     plt.figure(figsize=(25,int(25*row/col)))
#     for j in range(row*col):
#         plt.subplot(row,col,j+1)
#         plt.title(label[j].numpy())
#         plt.axis('off')
#         plt.imshow(img[j,])
#     plt.show()
#     break
# print(img.shape)

In [ ]:
# img.shape

In [ ]:
# row = 10; col = 8;
# row = min(row,config.BATCH_SIZE//col)
# N_TEST = count_data_items(test_files)
# print(N_TEST)
# ds = get_test_dataset(test_files)

# for (img,label) in ds:
#     plt.figure(figsize=(25,int(25*row/col)))
#     for j in range(row*col):
#         plt.subplot(row,col,j+1)
#         plt.title(label[j].numpy())
#         plt.axis('off')
#         plt.imshow(img[j,])
#     plt.show()
#     break
# print(img.shape)

## Model

In [ ]:
# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [ ]:
arr = [
         'blue_whale',
         'bottlenose_dolphin',
         'brydes_whale',
         'commersons_dolphin',
         'common_dolphin',
         'cuviers_beaked_whale',
         'dusky_dolphin',
         'false_killer_whale',
         'fin_whale',
         'frasiers_dolphin',
         'globis',
         'humpback_whale',
         'killer_whale',
         'long_finned_pilot_whale',
         'melon_headed_whale',
         'minke_whale',
         'pantropic_spotted_dolphin',
         'pilot_whale',
         'pygmy_killer_whale',
         'rough_toothed_dolphin',
         'sei_whale',
         'short_finned_pilot_whale',
         'spinner_dolphin',
         'spotted_dolphin',
         'white_sided_dolphin'
    
        ]



In [ ]:
# get adaptive margin
df_marg = pd.read_csv('/kaggle/input/happy-whale-and-dolphin/train.csv')
# df_marg = pd.read_csv('/kaggle/input/happywhale-splits/skf_species_10folds.csv')
# df_marg = df_marg[df_marg["species_x"].isin(arr)]
# df_marg.species.replace({"globis": "short_finned_pilot_whale",
#                           "pilot_whale": "short_finned_pilot_whale",
#                           "kiler_whale": "killer_whale",
#                           "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)
# df_marg = df_marg[df_marg["species"].isin(arr)]
tmp = np.sqrt(1 / np.sqrt(df_marg['individual_id'].value_counts().sort_index().values))
margins = (tmp - tmp.min()) / (tmp.max() - tmp.min()) * 0.55 + 0.05
# margins_remain = np.repeat(1e-9, config.N_CLASSES)
# margins = np.concatenate([margins, margins_remain], 0)[:config.N_CLASSES]
margins = margins.astype(np.float32)
margins.shape

In [ ]:
pd.DataFrame(margins).value_counts()

In [ ]:
# EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
#         efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

# def freeze_BN(model):
#     # Unfreeze layers while leaving BatchNorm layers frozen
#     for layer in model.layers:
#         if not isinstance(layer, tf.keras.layers.BatchNormalization):
#             layer.trainable = True
#         else:
#             layer.trainable = False

# # Function to create our EfficientNetB3 model
# def get_model():

#     if config.head=='arcface':
#         head = ArcMarginProduct
#     else:
#         assert 1==2, "INVALID HEAD"
    
#     with strategy.scope():
        
#         margin = head(
#             n_classes = config.N_CLASSES, 
#             s = 30, 
#             m = margins, #0.3, 
#             name=f'head/{config.head}', 
#             dtype='float32'
#             )

#         inp = tf.keras.layers.Input(shape = [config.IMAGE_SIZE, config.IMAGE_SIZE, 3], name = 'inp1')
#         label = tf.keras.layers.Input(shape = (), name = 'inp2')
        
#         if config.model_type == 'effnetv1':
#             x = EFNS[config.EFF_NET](weights = 'noisy-student', include_top = False)(inp)
#             embed = tf.keras.layers.GlobalAveragePooling2D()(x)
#             # inp = EFNS[config.EFF_NET](weights = 'noisy-student', include_top = False,input_shape = [config.IMAGE_SIZE, config.IMAGE_SIZE, 3])
#             # inp.layers[0]._name = 'inp1'
#             #inp.summary()
#             # x1=tf.keras.layers.GlobalAveragePooling2D()(inp.layers[-1].output)
#             # x2=tf.keras.layers.GlobalAveragePooling2D()(inp.layers[-5].output)
#             # x3=tf.keras.layers.GlobalAveragePooling2D()(inp.layers[-7].output)
#             # x4=tf.keras.layers.GlobalAveragePooling2D()(inp.layers[-13].output)
#             # embed =  tf.concat([x1,x2,x3,x4],axis = 1)

#         elif config.model_type == 'effnetv2':
#             FEATURE_VECTOR = f'{EFFNETV2_ROOT}/tfhub_models/efficientnetv2-{config.EFF_NETV2}/feature_vector'
#             embed = tfhub.KerasLayer(FEATURE_VECTOR, trainable=True)(inp)
            
#         embed = tf.keras.layers.Dropout(0.2)(embed)
#         # embed = tf.keras.layers.Dense(512)(embed)
#         embed = tf.keras.layers.Dense(512,kernel_regularizer=tf.keras.regularizers.l2(0.0001))(embed)
#         # embed = tf.keras.layers.Dense(config.N_CLASSES,kernel_regularizer=tf.keras.regularizers.l2(0.0001))(embed)       
#         # embed = tf.keras.layers.Dropout(0.3)(embed)
#         # embed = tf.keras.layers.Dense(2048)(embed)

#         x = margin([embed, label])
        
#         output = tf.keras.layers.Softmax(dtype='float32')(x)
        
#         model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
#         embed_model = tf.keras.models.Model(inputs = inp, outputs = embed)
#         # model = tf.keras.models.Model(inputs = [inp.input, label], outputs = [output])
#         # embed_model = tf.keras.models.Model(inputs = inp.input, outputs = embed)    
        
#         # opt = tf.keras.optimizers.Adam(learning_rate = config.LR)
#         # opt = tf.keras.optimizers.SGD(learning_rate=config.LR, momentum=0.9,decay=0.0005)
#         opt = tf.keras.optimizers.SGD(learning_rate=config.LR, momentum=0.9)
#         if config.FREEZE_BATCH_NORM:
#             freeze_BN(model)

#         model.compile(
#             optimizer = opt,
#             loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
#             metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
#             ) 
        
#         return model,embed_model

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def freeze_BN(model):
    # Unfreeze layers while leaving BatchNorm layers frozen
    for layer in model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

# Function to create our EfficientNetB3 model
def get_model():

    if config.head=='arcface':
        head = ArcMarginProduct
    else:
        assert 1==2, "INVALID HEAD"
    
    with strategy.scope():
        
        margin = head(
            n_classes = config.N_CLASSES, 
            s = 30, 
            m = margins, #0.3, 
            name=f'head/{config.head}', 
            dtype='float32'
            )

        # inp = tf.keras.layers.Input(shape = [config.IMAGE_SIZE, config.IMAGE_SIZE, 3], name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        
        if config.model_type == 'effnetv1':
            # x = EFNS[config.EFF_NET](weights = 'noisy-student', include_top = False)(inp)
            # embed = tf.keras.layers.GlobalAveragePooling2D()(x)
            inp = EFNS[config.EFF_NET](weights = 'noisy-student', include_top = False,input_shape = [config.IMAGE_SIZE, config.IMAGE_SIZE, 3])
            inp.layers[0]._name = 'inp1'
            embed = tf.keras.layers.GlobalAveragePooling2D()(inp.layers[-1].output)

        elif config.model_type == 'effnetv2':
            FEATURE_VECTOR = f'{EFFNETV2_ROOT}/tfhub_models/efficientnetv2-{config.EFF_NETV2}/feature_vector'
            embed = tfhub.KerasLayer(FEATURE_VECTOR, trainable=True)(inp.output)
            
        embed = tf.keras.layers.BatchNormalization()(embed) # batch norm or L2
        embed = tf.keras.layers.Dropout(0.2)(embed)
        embed = tf.keras.layers.Dense(512, kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0001))(embed)

        x = margin([embed, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)
        
        # model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
        # embed_model = tf.keras.models.Model(inputs = inp, outputs = embed)
        model = tf.keras.models.Model(inputs = [inp.input, label], outputs = [output])
        embed_model = tf.keras.models.Model(inputs = inp.input, outputs = embed)
        for layer in inp.layers:
          if isinstance(layer, tf.keras.layers.BatchNormalization):
              layer.trainable = False
        
        opt = tf.keras.optimizers.Adam(learning_rate = config.LR)
        # opt = tf.keras.optimizers.SGD(learning_rate=config.LR, momentum=0.9,decay=0.0005)
        # opt = tf.keras.optimizers.SGD(learning_rate=config.LR, momentum=0.9)
        if config.FREEZE_BATCH_NORM:
            freeze_BN(model)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5)]
            ) 
        
        return model,embed_model

In [ ]:
def get_lr_callback(plot=False):
    lr_start   = 0.0001 #0.000001
    lr_max     = 0.0002 #0.000002 * config.BATCH_SIZE  
    lr_min     = 0.000001 #0.000001
    lr_ramp_ep = 2
    lr_sus_ep  = 0
    lr_decay   = 0.96
   
    def lrfn(epoch):
        if config.RESUME:
            epoch = epoch + config.RESUME_EPOCH
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr
        
    if plot:
        epochs = list(range(config.EPOCHS))
        learning_rates = [lrfn(x) for x in epochs]
        plt.scatter(epochs,learning_rates)
        plt.show()

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

get_lr_callback(plot=True)

In [ ]:
class Snapshot(tf.keras.callbacks.Callback):
    
    def __init__(self,fold,snapshot_epochs=[]):
        super(Snapshot, self).__init__()
        self.snapshot_epochs = snapshot_epochs
        self.fold = fold
        
        
    def on_epoch_end(self, epoch, logs=None):
        # logs is a dictionary
#         print(f"epoch: {epoch}, train_acc: {logs['acc']}, valid_acc: {logs['val_acc']}")
        if epoch in self.snapshot_epochs: # your custom condition         
            self.model.save_weights(config.save_dir+f"/EF{config.MODEL_NAME}_epoch{epoch}.h5")
        self.model.save_weights(config.save_dir+f"/{config.MODEL_NAME}_last.h5")

## Train

In [ ]:
# # TRAINING_FILENAMES = [x for i,x in enumerate(train_files) if i%config.FOLDS!=config.FOLD_TO_RUN]
# # VALIDATION_FILENAMES = [x for i,x in enumerate(train_files) if i%config.FOLDS==config.FOLD_TO_RUN]
# val_filename = GCS_DS_PATH + "/happywhale-2022-train-10-645.tfrec"
# TRAINING_FILENAMES = [x for i,x in enumerate(train_files)]
# TRAINING_FILENAMES.remove(val_filename)
# VALIDATION_FILENAMES = [val_filename]
# print(len(TRAINING_FILENAMES),len(VALIDATION_FILENAMES),count_data_items(TRAINING_FILENAMES),count_data_items(VALIDATION_FILENAMES))
# PSEUDO_FILENAMES = [x for i,x in enumerate(pseudo_files)]
# print(len(PSEUDO_FILENAMES),count_data_items(PSEUDO_FILENAMES))
TRAINING_FILENAMES = [x for i,x in enumerate(train_files)]
VALIDATION_FILENAMES = []
PSEUDO_FILENAMES = [x for i,x in enumerate(pseudo_files)]
print(len(TRAINING_FILENAMES),len(VALIDATION_FILENAMES),count_data_items(TRAINING_FILENAMES),count_data_items(VALIDATION_FILENAMES))
print(len(PSEUDO_FILENAMES),count_data_items(PSEUDO_FILENAMES))


In [ ]:
TRAINING_FILENAMES = TRAINING_FILENAMES+PSEUDO_FILENAMES
print(len(TRAINING_FILENAMES),len(VALIDATION_FILENAMES),count_data_items(TRAINING_FILENAMES),count_data_items(VALIDATION_FILENAMES))

In [ ]:
TRAINING_FILENAMES

In [ ]:
VALIDATION_FILENAMES

In [ ]:
if config.DEBUG:
    TRAINING_FILENAMES = [TRAINING_FILENAMES[0]]
    VALIDATION_FILENAMES = [VALIDATION_FILENAMES[0]]
    print(len(TRAINING_FILENAMES),len(VALIDATION_FILENAMES),count_data_items(TRAINING_FILENAMES),count_data_items(VALIDATION_FILENAMES))
    test_files = [test_files[0]]

In [ ]:
seed_everything(config.SEED)
VERBOSE = 1
train_dataset = get_training_dataset(TRAINING_FILENAMES)
val_dataset = get_val_dataset(VALIDATION_FILENAMES)
STEPS_PER_EPOCH = count_data_items(TRAINING_FILENAMES) // config.BATCH_SIZE
train_logger = tf.keras.callbacks.CSVLogger(config.save_dir+'/training-log-fold-%i.h5.csv'%config.FOLD_TO_RUN)
# SAVE BEST MODEL EACH FOLD        
sv_loss = tf.keras.callbacks.ModelCheckpoint(
    config.save_dir+f"/{config.MODEL_NAME}_loss.h5", monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='min', save_freq='epoch')
# BUILD MODEL
K.clear_session()
model,embed_model = get_model()
snap = Snapshot(fold=config.FOLD_TO_RUN,snapshot_epochs=[10,15,16,17,18,19,20,21,24,27,30,35,40,45,48,53,57])
model.summary()

# if config.RESUME:   
#     model.load_weights(config.resume_model_wts)

In [ ]:
import gc
gc.collect()

In [ ]:
#use body data set pretrain model
# model.load_weights('/kaggle/input/pretrainedbodyb7ep21/EFeffnetv1_b7_epoch21.h5')
model.load_weights('/kaggle/input/bodyvfinal/effnetv1_b5_loss.h5')


In [ ]:
print('#### Image Size %i with EfficientNet B%i and batch_size %i'%
      (config.IMAGE_SIZE,config.EFF_NET,config.BATCH_SIZE))

history = model.fit(train_dataset,
                validation_data = val_dataset,
                steps_per_epoch = STEPS_PER_EPOCH,
                epochs = config.EPOCHS,
                callbacks = [snap,get_lr_callback(),train_logger,sv_loss], 
                verbose = VERBOSE)